In [30]:
from gensim.models.coherencemodel import CoherenceModel
import math
from gensim.corpora.dictionary import Dictionary
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [68]:
test_texts = [
    ["hallo","ich","bin", "robin", "ist"],
    ["mein","name","ist"],
    ["robin", "name","mein"]
]

dic = Dictionary(test_texts)

In [13]:
dict(dic)

{0: 'bin', 1: 'hallo', 2: 'ich', 3: 'robin', 4: 'ist', 5: 'mein', 6: 'name'}

In [122]:
cm = CoherenceModel(topics=[["robin","grütze"],["name","ist"]], 
                    texts= test_texts,
                    coherence='c_npmi',  
                    dictionary=dic)

coherence_per_topic = cm.get_coherence_per_topic()

coherence_per_topic

/Users/robinfeldmann/Library/Python/3.11/lib/python/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/robinfeldmann/Library/Python/3.11/lib/python/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[nan, -0.26185950714089923]

In [119]:
def w(word):
    return len([x for x in test_texts if word in x])/len(test_texts)
def ww(x,y):
    return len([text for text in test_texts if x in text and y in text])/len(test_texts)

def npmi(x,y):
    klein = float(w(x)) *float(w(y))
    zähler = math.log(  ww(x,y)/klein)
    nenner = (-math.log(ww(x,y))) + 0.0001
    return zähler/nenner

def topic_npmi(topic):
    val = 0
    for i in range(len(topic)-1):
        for k in range(i+1,len(topic) ):
            val += npmi(topic[i],topic[k])
    return val/len(topic)

In [120]:
topic_npmi(["bin","mein"])

ValueError: math domain error

In [114]:
topic_npmi(['name','ist'])

-0.13092975357145742

In [107]:
npmi("robin","mein")

robin mein


-0.26185950714291484

$$\frac{ln(\frac{p(x,y)}{p(x)p(y)})}{-ln(p(x,y))}$$